In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
#Loading Dataset
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [64]:
#Setup
n_nodes_hl1 = 7
n_nodes_hl2 = 30
n_nodes_hl3 = 30

n_classes = 10

batch_size = 100

x = tf.placeholder('float',[None,784])
y = tf.placeholder('float')

def neural_network_model(data):
    hidden_1_layer = {'weights':tf.Variable(tf.random_normal([784, n_nodes_hl1])),
                     'biases':tf.Variable(tf.random_normal([n_nodes_hl1]))}
    
    hidden_2_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl1, n_nodes_hl2])),
                     'biases':tf.Variable(tf.random_normal([n_nodes_hl2]))}
    
    hidden_3_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl2, n_nodes_hl3])),
                     'biases':tf.Variable(tf.random_normal([n_nodes_hl3]))}
    
    output_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl3, n_classes])),
                     'biases':tf.Variable(tf.random_normal([n_classes]))}
    
    
    l1 = tf.add(tf.matmul(data,hidden_1_layer['weights']), hidden_1_layer['biases'])
    l1 = tf.nn.relu(l1)
    
    l2 = tf.add(tf.matmul(l1,hidden_2_layer['weights']), hidden_2_layer['biases'])
    l2 = tf.nn.relu(l2)
    
    l3 = tf.add(tf.matmul(l2,hidden_3_layer['weights']), hidden_3_layer['biases'])
    l3 = tf.nn.relu(l3)
    
    output = tf.add(tf.matmul(l3,output_layer['weights']), output_layer['biases'])
    output = tf.nn.relu(output)
    
    return output

In [65]:
class NeuralNetwork:
    def __init__(self,sizes):
        self.layers = []
        
        
        for i in range(1,len(sizes)):
            self.layers.append(
               {'weights':tf.Variable(tf.random_normal([sizes[i-1], sizes[i]])),
                         'biases':tf.Variable(tf.random_normal([sizes[i]]))} 
            )
    
    def model(self,data):
        l = tf.add(tf.matmul(data,self.layers[0]['weights']), self.layers[0]['biases'])
        l = tf.nn.relu(l)
        for i in range(1,len(self.layers)):
            l = tf.add(tf.matmul(l,self.layers[i]['weights']), self.layers[i]['biases'])
            l = tf.nn.relu(l)
        return l
    
    def train(self,x,hm_epochs=10):
        prediction = self.model(x)
        cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(prediction,y))

        #learning rate = 0.001
        optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)

        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())

            for epoch in range(hm_epochs):
                epoch_loss = 0

                for _ in range(int(mnist.train.num_examples/batch_size)):
                    epoch_x,epoch_y = mnist.train.next_batch(batch_size)
                    _, c = sess.run([optimizer, cost], feed_dict = {x: epoch_x, y: epoch_y})
                    epoch_loss += c

                print('Epoch',epoch+1, 'completed out of',hm_epochs,'loss:', epoch_loss)
                
            self.prediction = prediction

                
            
    def score (self,features,labels):
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            correct = tf.equal(tf.argmax(self.prediction,1),tf.argmax(y,1))
            accuracy = tf.reduce_mean(tf.cast(correct,'float'))
            print('Accuracy: ',accuracy.eval({x:features, y:labels}))

In [66]:
def train_neural_network(x):
    prediction = neural_network_model(x)
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(prediction,y))
    
    #learning rate = 0.001
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    
    hm_epochs = 10
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        
        for epoch in range(hm_epochs):
            epoch_loss = 0
            
            for _ in range(int(mnist.train.num_examples/batch_size)):
                epoch_x,epoch_y = mnist.train.next_batch(batch_size)
                _, c = sess.run([optimizer, cost], feed_dict = {x: epoch_x, y: epoch_y})
                epoch_loss += c
            
            print('Epoch',epoch+1, 'completed out of',hm_epochs,' loss:', epoch_loss)
        
        
        correct = tf.equal(tf.argmax(prediction,1),tf.argmax(y,1))
        accuracy = tf.reduce_mean(tf.cast(correct,'float'))
        print('Accuracy: ',accuracy.eval({x:mnist.test.images, y:mnist.test.labels}))

In [67]:
#train_neural_network(x)
nn = NeuralNetwork([784,7,10])
nn.train(x,30)
nn.score(mnist.test.images, mnist.test.labels)

Epoch 1 completed out of 30 loss: 12.5998153687
Epoch 2 completed out of 30 loss: 11.259926796
Epoch 3 completed out of 30 loss: 9.11562919617
Epoch 4 completed out of 30 loss: 10.5394124985
Epoch 5 completed out of 30 loss: 10.5108060837
Epoch 6 completed out of 30 loss: 10.2783241272
Epoch 7 completed out of 30 loss: 10.8187179565
Epoch 8 completed out of 30 loss: 9.36462211609
Epoch 9 completed out of 30 loss: 9.39374542236
Epoch 10 completed out of 30 loss: 10.3444852829
Epoch 11 completed out of 30 loss: 8.99594593048
Epoch 12 completed out of 30 loss: 8.44401264191
Epoch 13 completed out of 30 loss: 9.77537918091
Epoch 14 completed out of 30 loss: 9.19965457916
Epoch 15 completed out of 30 loss: 10.1312332153
Epoch 16 completed out of 30 loss: 10.3272237778
Epoch 17 completed out of 30 loss: 8.72772598267
Epoch 18 completed out of 30 loss: 8.45911407471
Epoch 19 completed out of 30 loss: 9.07177639008
Epoch 20 completed out of 30 loss: 8.45788002014
Epoch 21 completed out of 30 l